In [2]:
import requests
import time
import datetime
import pandas as pd
import sys
import string
import pyodbc

In [3]:
def scraper(ticker, latest_xhr_id='268864272', max_volume=50000, start_date='2020-01-01', end_date='2020-12-31'):

    
    inv_table = {}
    
    # Push Errors if scrape volume less than 0:
    if max_volume <= 0:
        sys.exit("Error: max_volume must be more than 0")

    start = time.time()
    master_content = []  # List to store all data extracted
    scroll_list = [latest_xhr_id]  # List to store all XHR id to be part of the url parameters
    tracker_list = []  # List containing integers for tracking progress
    tracker = 0
    fail_count = 0

    for x in range(5001):
        if x > 0:
            addition = x * 100
            tracker_list.append(addition)

    # Running for loop for collecting data from stocktwits. Each loop collects 20 comments.
    for _ in range(max_volume):
        try:
            headers = {
                'authority': 'api.stocktwits.com',
                'accept': 'application/json',
                'authorization': 'OAuth 6439333424451d1c85e731fb126006f7780192d2',
                'user-agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                               'Chrome/87.0.4280.88 Safari/537.36'),
                'origin': 'https://stocktwits.com',
                'sec-fetch-site': 'same-site',
                'sec-fetch-mode': 'cors',
                'sec-fetch-dest': 'empty',
                'referer': 'https://stocktwits.com/',
                'accept-language': 'en-US,en;q=0.9',
            }

            params = (
                ('symbols',ticker),
                ('filter', 'all'),
                ('limit', '30'),
                ('max', scroll_list[-1]),
                ('since', 224610272)
            )

            response = requests.get(f'https://api.stocktwits.com/api/2/streams/symbols.json',
                                    headers=headers, params=params)
            content = response.json()
            messages = content['messages']
            # Creating dictionary for items scraped
            for item in messages:
                content_dict = {}
                content_dict['Doc_id'] = item['id']
                content_dict['Message'] = item['body']
                content_dict['Date'] = item['created_at'].split('T')[0]
                content_dict['Time'] = item['created_at'].split('T')[1]
                content_dict['Symbols'] = []
                content_dict['Username'] = item['user']['username']
                content_dict['Name'] = item['user']['name']
                
                
                try:
                    content_dict['Sentiment'] = item['entities']['sentiment']['basic']
                except TypeError:
                    content_dict['Sentiment'] = "N/A"
                    
                for i in range(len(item['symbols'])):
                    label = item['symbols'][i]['symbol']
                    content_dict['Symbols'].append(label)
                    
                    if label in inv_table:
                        inv_table[label].append(content_dict['Doc_id'])
                    else:
                        inv_table[label] = [content_dict['Doc_id']]

                master_content.append(content_dict.copy())
                    

            next_20_id = str(messages[-1]['id'])
            scroll_list.append(next_20_id)

            # Progress Tracker
            tracker += 1

            # Variables for tracker
            last_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')
            first_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')

            diff = last_date - first_date

            three_quarter_done = first_date + diff/4
            half_done = first_date + diff/2
            one_quarter_done = first_date + diff*3/4

            # Trackers
            for number in tracker_list:
                if tracker == number:
                    print(f"Extracted {number}...")
                    print(f"run time = {time.time() - start}")  # Check run time

            if (master_content[-1]['Time'].split(":")[0] == "00" and
                    master_content[-1]['Date'] == f'{one_quarter_done}'):
                print("25% done")

            elif (master_content[-1]['Time'].split(":")[0] == "00" and
                    master_content[-1]['Date'] == f'{half_done}'):
                print("50% done")

            elif (master_content[-1]['Time'].split(":")[0] == "00" and
                    master_content[-1]['Date'] == f'{three_quarter_done}'):
                print("75% done")

            elif (master_content[-1]['Time'].split(":")[0] == "00" and
                    master_content[-1]['Date'] == f'{first_date}'):
                print("100% done")
                print(f'Number of tweets unable to scrape: {fail_count * 20}')
                break

        except:
            fail_count += 1

    print(f"Number of tweets scraped: {len(master_content)}")
    print(f"Last Tweet: {master_content[-1]}")

    df = pd.DataFrame(master_content)
    
    for i in inv_table:
        inv_table[i].sort()
        
    return df,inv_table


if __name__ == "__main__":
    tweets_df,inv_table = scraper ("AAPL,BABA",max_volume = 2)
    

Number of tweets scraped: 60
Last Tweet: {'Doc_id': 268849354, 'Message': '$BABA the worst stock I have ever invested my money in.', 'Date': '2020-12-31', 'Time': '22:40:51Z', 'Symbols': ['BABA'], 'Username': 'jccoo', 'Name': 'James Cooper', 'Sentiment': 'N/A'}


In [ ]:
#to update the database 
tweets_df.to_csv('tweets.csv')

# Tokenize, Filter and Lemmatize

In [14]:
tweets_df = pd.read_csv('Database(1).csv')
tweets = tweets_df.copy()
tweets

,Unnamed: 0,Doc_id,Message,Date,Time,Symbols,Sentiment,Words
0,0,268863929,AAPL SPY QQQ Will AAPL hit 3 trillion market ...,2020-12-31,23:58:49Z,"['AAPL', 'SPY', 'QQQ']",Bullish,"['aapl', 'spy', 'qqq', 'will', 'aapl', 'hit', ..."
1,1,268863920,AAPL so you would think it was a perfect day ...,2020-12-31,23:58:40Z,['AAPL'],Bullish,"['aapl', 'would', 'think', 'perfect', 'day', '..."
2,2,268863653,AAPL buy price Apple for me 64 can t wait to ...,2020-12-31,23:56:52Z,['AAPL'],Bullish,"['aapl', 'buy', 'price', 'apple', '64', 'wait'..."
3,3,268863594,AAPL can i get a hi 5,2020-12-31,23:56:31Z,['AAPL'],NaN,"['aapl', 'get', 'hi', '5']"
4,4,268862960,AAPL Happy New Year By Melbenross Apple,2020-12-31,23:52:32Z,['AAPL'],Bullish,"['aapl', 'happy', 'new', 'year', 'by', 'melben..."
5,5,268862897,AAPL hard to see now eventually the bleeding ...,2020-12-31,23:52:10Z,['AAPL'],NaN,"['aapl', 'hard', 'see', 'eventually', 'bleedin..."
6,6,268862612,AMZN I think this symmetric triangle and all ...,2020-12-31,23:50:08Z,"['AAPL', 'AMZN', 'SPY', 'TSLA', 'QQQ']",Bullish,"['amzn', 'i', 'think', 'symmetric', 'triangle'..."
7,7,268862114,AAPL Anyone stressed about this stock needs t...,2020-12-31,23:46:55Z,['AAPL'],Bullish,"['aapl', 'anyone', 'stressed', 'stock', 'need'..."
8,8,268861567,AAPL to All and Everyone have a wonderful New...,2020-12-31,23:43:44Z,['AAPL'],Bullish,"['aapl', 'all', 'everyone', 'wonderful', 'new'..."
9,9,268860519,AAPL,2020-12-31,23:37:56Z,['AAPL'],NaN,['aapl']


In [161]:
import nltk
#nltk.download()
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams

stop_words = set(stopwords.words("english"))

lem = WordNetLemmatizer()

In [162]:
company_ticker = {"baba":"alibaba",
                  "amzn":"amazon",
                  "aapl":"apple",
                  "tsla":"tesla",
                  "msft":"microsoft",
                  "fb":"facebook",
                  "googl":"google",
                  "nio":"nio",
                  "twtr":"twitter",
                  "nflx":"netflix"
                 }

In [163]:
def token_filter_lemmatize(msg):
    words = word_tokenize(msg)
    
    processed = []
    
    for w in words:
        if (w not in stop_words) and (w not in string.punctuation):
            if (w in company_ticker):
                w = company_ticker[w]
            processed.append(lem.lemmatize(w.lower()))
            
    return processed

tweets['Words'] = tweets['Message'].apply(token_filter_lemmatize)

#tweets.to_csv(r'Database.csv', index=False)
tweets

,Unnamed: 0,Doc_id,Message,Date,Time,Symbols,Sentiment,Words
0,0,268863929,AAPL SPY QQQ Will AAPL hit 3 trillion market ...,2020-12-31,23:58:49Z,"['AAPL', 'SPY', 'QQQ']",Bullish,"[aapl, spy, qqq, will, aapl, hit, 3, trillion,..."
1,1,268863920,AAPL so you would think it was a perfect day ...,2020-12-31,23:58:40Z,['AAPL'],Bullish,"[aapl, would, think, perfect, day, buy, call, ..."
2,2,268863653,AAPL buy price Apple for me 64 can t wait to ...,2020-12-31,23:56:52Z,['AAPL'],Bullish,"[aapl, buy, price, apple, 64, wait, see, new, ..."
3,3,268863594,AAPL can i get a hi 5,2020-12-31,23:56:31Z,['AAPL'],NaN,"[aapl, get, hi, 5]"
4,4,268862960,AAPL Happy New Year By Melbenross Apple,2020-12-31,23:52:32Z,['AAPL'],Bullish,"[aapl, happy, new, year, by, melbenross, apple]"
5,5,268862897,AAPL hard to see now eventually the bleeding ...,2020-12-31,23:52:10Z,['AAPL'],NaN,"[aapl, hard, see, eventually, bleeding, stop, ..."
6,6,268862612,AMZN I think this symmetric triangle and all ...,2020-12-31,23:50:08Z,"['AAPL', 'AMZN', 'SPY', 'TSLA', 'QQQ']",Bullish,"[amzn, i, think, symmetric, triangle, bullshit..."
7,7,268862114,AAPL Anyone stressed about this stock needs t...,2020-12-31,23:46:55Z,['AAPL'],Bullish,"[aapl, anyone, stressed, stock, need, stop, bu..."
8,8,268861567,AAPL to All and Everyone have a wonderful New...,2020-12-31,23:43:44Z,['AAPL'],Bullish,"[aapl, all, everyone, wonderful, new, year, se..."
9,9,268860519,AAPL,2020-12-31,23:37:56Z,['AAPL'],NaN,[aapl]


# Inverted_Index and Search

In [164]:
# Create Inverted Index

# FORMAT (Preparing for JSON):

# Inv_Index -> Words -> Documents -> Position + Tf.idf

inverted_index = {}

docIndex = tweets.columns.get_loc("Doc_id")
wordsIndex = tweets.columns.get_loc("Words")

for index, doc in tweets.iterrows():
    docID = doc[docIndex]
    words = doc[wordsIndex]
    
    i = 0
    
    for w in words:

        if w in company_ticker:
            w = company_ticker[w]
        
        if w not in inverted_index:
            inverted_index[w] = {}
        
        if docID not in inverted_index[w]:
            inverted_index[w][docID] = {'pos': [i], 'tf.idf':0}
        else:
            inverted_index[w][docID]['pos'].append(i)

        i+=1

In [175]:
# CALCULATE TF.IDF
## TF = (count of word in doc)/(total words in doc)
## IDF = (Total number of Docs)/(Number of docs containing word)

import math
tweets_docid = tweets.set_index("Doc_id")['Words']
N = len(tweets_docid)

def idf(word_dict):
    value = N/len(word_dict)
    return math.log(value, 10)


def tf(word_dict, docID):
    
    count = len(word_dict[docID]['pos'])
    total = len(tweets_docid[docID])
    
    return (count/total)

for word in inverted_index:
    idf_value = idf(inverted_index[word])
    
    for docID in inverted_index[word]:
        tf_value = tf(inverted_index[word], docID)
        inverted_index[word][docID]['tf.idf'] = tf(inverted_index[word], docID) * idf_value
        
inverted_index

{'apple': {268863929: {'pos': [0, 4, 19], 'tf.idf': 0.0},
  268863920: {'pos': [0], 'tf.idf': 0.0},
  268863653: {'pos': [0, 3, 10], 'tf.idf': 0.0},
  268863594: {'pos': [0], 'tf.idf': 0.0},
  268862960: {'pos': [0, 6], 'tf.idf': 0.0},
  268862897: {'pos': [0], 'tf.idf': 0.0},
  268862612: {'pos': [16], 'tf.idf': 0.0},
  268862114: {'pos': [0], 'tf.idf': 0.0},
  268861567: {'pos': [0], 'tf.idf': 0.0},
  268860519: {'pos': [0], 'tf.idf': 0.0},
  268860466: {'pos': [2], 'tf.idf': 0.0},
  268858750: {'pos': [0], 'tf.idf': 0.0},
  268858597: {'pos': [0], 'tf.idf': 0.0},
  268858043: {'pos': [1], 'tf.idf': 0.0},
  268857368: {'pos': [7], 'tf.idf': 0.0},
  268856542: {'pos': [15], 'tf.idf': 0.0},
  268856483: {'pos': [3], 'tf.idf': 0.0},
  268856237: {'pos': [0], 'tf.idf': 0.0},
  268855704: {'pos': [0], 'tf.idf': 0.0},
  268855376: {'pos': [0], 'tf.idf': 0.0},
  268855171: {'pos': [3], 'tf.idf': 0.0},
  268854585: {'pos': [0], 'tf.idf': 0.0},
  268854050: {'pos': [2], 'tf.idf': 0.0},
  2688

In [176]:
import json

def store_inv_index():
    with open('InvIndex.json', 'w') as outfile:
        json.dump(inverted_index,outfile)

store_inv_index()

# Querying the database

In [25]:
from flask import Flask
from flask_restful import Api, Resource
from flask import request
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams
import pyodbc
import json
from collections import defaultdict
import math


app = Flask(__name__)
api = Api(app)

# conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=DESKTOP-97U22LI;DATABASE=CZ4034;UID=sa;PWD=password1')
# cursor = conn.cursor()

stop_words = set(stopwords.words("english"))
lem = WordNetLemmatizer()

class SearchResource(Resource):    
    def __init__(self):
        self.InvIndex = defaultdict(set, self.getInvIndex())
        self.words = pd.Series(list(self.InvIndex.keys()))
        self.company_ticker = {"baba":"alibaba",
                              "amzn":"amazon",
                              "aapl":"apple",
                              "tsla":"tesla",
                              "msft":"microsoft",
                              "fb":"facebook",
                              "googl":"google",
                              "nio":"nio",
                              "twtr":"twitter",
                              "nflx":"netflix"
                             }
        
        # CHANGE PARAM WHEN MORE SCRAPED!
        self.num_docs = 500
        
    ################# API FUNCTIONS #######################
        
    def get(self):
        query = request.args.get('query')
        matched_index = self.search(query)
        
        data = self.queryDB('tweets',matched_index).to_json() 
        
        return {'content': data}
    
    def post(self):
        return {"data": "posted!"}
    
    ################# QUERY SEARCH #######################
    
    def clean_query(self, query, exact):
        query_clean = []
        
        query_split = word_tokenize(query)
        
        for query_word in query_split:
            word_lower = query_word.lower()
            if word_lower not in stop_words:
                cur = lem.lemmatize(word_lower)
                
                if cur in self.company_ticker:
                    cur = self.company_ticker[cur]
                    
                if not exact:
                    cur = self.JDreco(cur)
                
                if (cur not in query_clean):
                    query_clean.append(cur)
                    
        return query_clean
    
    def search(self, query, exact = False, filter_on = False, filter_amt = 10):
        matched_documents = None
        
        # (1) Clean Query:
        query_clean = self.clean_query(query, exact)
        print("Recommended Query List:")
        print(query_clean)
        
        
        # (2) Find All Docs (match any one word in query):
        for word in query_clean:
            if word in self.InvIndex:
                matches = self.InvIndex[word]
                match_index = list(matches.keys())
            else:
                print("No Matches for Word: "+word)
                continue
    
            if matched_documents is None:
                matched_documents = match_index
            else:
                #matched_documents = list(set.intersection(set(matched_documents),set(match_index)))
                matched_documents = list(set.union(set(matched_documents),set(match_index)))
                
                
        # (3) Filter Most Relevant:
        if filter_on:
            weights = self.tf_idf(query_clean)
            
            doc_score = {}
            
            for docID in matched_documents:
                doc_score[docID] = self.calc_match_score(weights, docID)
                
            matched_documents = sorted(doc_score, key=doc_score.get, reverse = True)[:filter_amt]
        
        # (4) Return Results:
        return matched_documents
    
    ################# RECOMMENDATION #######################

    def jaccard(self,entry, gram_number):
        spellings = self.words[self.words.str.startswith(entry[0])]
        distances = ((jaccard_distance(set(ngrams(entry,gram_number)),
                                           set(ngrams(word,gram_number))), word)
                     for word in spellings)
        closest = min(distances)
        return closest[1]
    
    
    def JDreco(self,entry):
        return self.jaccard(entry, 2)
    
    ################# TF.IDF #######################
    
    def tf_idf(self,query):
        weights = {}
        
        for word in query:
            
            # Find IDF value
            if (word in self.InvIndex):
                idf_value = self.idf(self.InvIndex[word])
            else:
                idf_value = 0
                
            # Find TF Value
            tf_value = 1/len(query)
            
            # Store TF.IDF
            weights[word] = tf_value * idf_value
        return weights
            
    
    def idf(self,word_dict):
        value = self.num_docs/len(word_dict)
        return math.log(value, 10)
    
    ################# FILTER MOST RELEVANT #######################
    
    def calc_match_score(self, weights_query, docID):
        
        result = 0
        
        for word in weights_query:

            #QUERY
            query_score = weights_query[word]
            
            #DOC
            try:
                doc_score = float(self.InvIndex[word][docID]['tf.idf'])
            except:
                doc_score = 0
            
            #PRODUCT
            result += query_score * doc_score
            
        return result
    
    ################# INVERTED INDEX #######################
    
    def getInvIndex(self):
        with open('InvIndex.json') as infile:
            data = json.load(infile)
        return data
    
    ################# DATABASE CONNECTION & QUERY #######################

    def queryDB(self,table, dbQuery):
        return pd.read_sql_query('SELECT * FROM CZ4034.dbo.'+table+' where Doc_id IN '+str(tuple(dbQuery)),conn)
    
    def queryOffline(self, query, exact = False, filter_on = False, filter_amt = 10):
        match_list = self.search(query, exact = exact, filter_on = filter_on, filter_amt = filter_amt)
        
        for docID in match_list:
            print(tweets[tweets['Doc_id']==int(docID)]['Message'])

api.add_resource(SearchResource, "/search" )

In [37]:
x = SearchResource()
x.queryOffline("hereby declare today", filter_on = True)

Recommended Query List:
['here', 'dec', 'today']
484     AAPL On my second best day of 2020 I made 50 ...
Name: Message, dtype: object
337     AAPL when does after hours close today 
Name: Message, dtype: object
282     AAPL can we actually see this finish in green...
Name: Message, dtype: object
419     AAPL My guess is it finishes around 131 90 today
Name: Message, dtype: object
28     DJIA AAPL FB TSLA WISHING YOU ALL HERE A HAPP...
Name: Message, dtype: object
455     BNGO NOT A PUMP AND DUMP RETIREMENT PLAN RIGH...
Name: Message, dtype: object
355     AAPL finish 137 today to see 145 eow next week
Name: Message, dtype: object
374     SPY AAPL TSLA NIO At what time does the marke...
Name: Message, dtype: object
383    Top Bullish Flow Today TSLA AAPL NIO AAL QS
Name: Message, dtype: object
218     AAPL Such a beautiful curl today looks fantas...
Name: Message, dtype: object


# Separating Train Data for Model

In [ ]:
train = tweets[['Sentiment', 'Words']]
train
train.to_csv(r'Train.csv', index=False)

In [93]:
from collections import defaultdict

inverted_index = defaultdict(set)
# bigram_inverted_index = defaultdict(set)

docIndex = tweets.columns.get_loc("Doc_id")
wordsIndex = tweets.columns.get_loc("Words")

for index, doc in tweets.iterrows():
    docID = doc[docIndex]
    words = doc[wordsIndex]
    
    i = 0
#     first = True
    
    for w in words:
        
        if w in company_ticker:
            w = company_ticker[w]
        
        inverted_index[w].add((docID,i))
        
#         if not first:
#             bigram = prev+' '+cur
#             bigram_inverted_index[bigram].add((docID,i-1))
            
#         first = False
#         prev = cur

        i+=1
    
inverted_index

defaultdict(set,
            {'apple': {(268748698, 6),
              (268748881, 0),
              (268748891, 0),
              (268748948, 4),
              (268749062, 0),
              (268749071, 0),
              (268749137, 0),
              (268749411, 0),
              (268749503, 0),
              (268749776, 0),
              (268749819, 5),
              (268749841, 0),
              (268749866, 0),
              (268750001, 0),
              (268750001, 1),
              (268750217, 0),
              (268750451, 0),
              (268750573, 0),
              (268750803, 0),
              (268750803, 5),
              (268750910, 0),
              (268751182, 17),
              (268751293, 0),
              (268751721, 0),
              (268751913, 0),
              (268751913, 13),
              (268752190, 0),
              (268752557, 0),
              (268752745, 2),
              (268752899, 0),
              (268752899, 3),
              (268753025, 0),
            

In [94]:
available_words = pd.Series(list(inverted_index.keys()))

def search(query, df = tweets):
    matched_documents = set()
    
    words = word_tokenize(query)
    
    prev = None
    
    for word in words:
        word_lower = word.lower()
        if word_lower not in stop_words:
            cur = lem.lemmatize(word_lower)
            
            reco_cur = JDreco(cur)
            print(reco_cur)
            
            match1 = inverted_index.get(reco_cur)

            if match1:
                # The operator |= is a short hand for set union
                matched_documents |= match1
                
    match_index = [item[0] for item in matched_documents]
    return df[df['Doc_id'].isin(match_index)]


def jaccard(entry, gram_number):
    spellings = available_words[available_words.str.startswith(entry[0])]
    distances = ((jaccard_distance(set(ngrams(entry,gram_number)),
                                       set(ngrams(word,gram_number))), word)
                 for word in spellings)

    closest = min(distances)
    return closest[1]


def JDreco(entry):
    return jaccard(entry, 2)

In [58]:
def search(query, df = tweets, exact = False):
    matched_documents = set()
    
    words = word_tokenize(query)
    
    prev = None
    
    for word in words:
        word_lower = word.lower()
        if word_lower not in stop_words:
            cur = lem.lemmatize(word_lower)
            
            if cur in company_ticker:
                cur = company_ticker[cur]
            
            reco_cur = JDreco(cur)
            
            matches = inverted_index.get(reco_cur)

            if matches:
                # The operator |= is a short hand for set union
                matched_documents |= matches
                
    
                
    match_index = [item[0] for item in matched_documents]
    return match_index #df[df['Doc_id'].isin(match_index)] 